In [4]:
'''Init and connector'''
from base.helper import *
from base.staad_base.geometry import *
from base.staad_base.load import *
from base.staad_base.design import *
from base.staad_base.property import *
from base.staad_base.optimise_member import *
from IPython.display import display_markdown
import pyperclip

openSTAAD,STAAD_objects = get_openSTAAD()

geometry = STAAD_objects.geometry
property = STAAD_objects.property
output = STAAD_objects.output

beam_nos = get_beam_nos(geometry)

In [5]:
'''Selection of members -> array to form group'''
selected_members = get_selected_beam_nos(geometry)
pyperclip.copy(f'{selected_members}')
print(f'copied {len(selected_members)} members -> {selected_members}')

copied 62 members -> [24, 25, 36, 56, 57, 79, 80, 124, 125, 169, 170, 214, 215, 259, 260, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 418, 421, 422, 423, 426, 427, 428, 430, 431, 434, 435, 437, 438, 491, 493, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 511, 512, 513, 514, 515]


In [ ]:
'''Member groups'''

'''BEAMS'''
beams_tier_1 = member_group(
    id='tier_1',
    members=[37, 38, 41, 42, 43, 46, 47, 57, 60, 62, 65],
    profiles=[11,12],
    Staad_objects= STAAD_objects
)

beams_tier_2 = member_group(
    id='tier_2',
    members=[111, 112, 115, 116, 117, 120, 121, 130],
    profiles=[7,9,11],
    Staad_objects= STAAD_objects
)

int_beams_tier_2 = member_group(
    id='tier_2',
    members=[319, 322, 325, 328, 331, 334, 348, 349],
    profiles=[10,9],
    Staad_objects= STAAD_objects
)

beams_tier_3 = member_group(
    id='tier_3',
    members=[169, 170, 171, 172, 173, 174, 175, 182, 183, 184, 185, 186, 187, 188, 199, 200, 201, 202, 203, 204, 205, 214, 215, 216, 217, 218, 219, 220, 227, 228, 229, 230, 231, 232, 233, 240, 241, 242, 243, 244, 245, 246, 257, 258, 259, 260, 261, 262, 263, 270, 271, 272, 273, 274, 275, 276],
    profiles=[17,7,9],
    Staad_objects= STAAD_objects
)

tie_beams_group_1 = member_group(
    id='tie_beams_1',
    members=[81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 163, 164, 165, 166, 167, 168, 277, 278, 279, 280, 281, 282, 283, 284, 286, 288, 289, 291, 293, 294, 296, 297, 299, 301, 302, 303, 304, 305, 306, 307, 308, 309],
    profiles=[2,11],
    Staad_objects= STAAD_objects
)

'''COLUMNS'''
columns_group_1 = member_group(
    id='main_cols',
    members=[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162],
    profiles=[9,11,12],
    Staad_objects=STAAD_objects
)

stub_columns_group_1 = member_group(
    id='stub_cols',
    members=[285, 287, 290, 292, 295, 298, 300, 310, 312, 313, 314, 315, 316, 317, 320, 321, 323, 324, 326, 327, 329, 330, 332, 333],
    profiles=[6,13,14],
    Staad_objects=STAAD_objects
)

'''BRACINGS'''

plan_bracing = member_group(
    id='plan_bracing',
    members=[39, 40, 44, 45, 58, 59, 63, 64, 113, 114, 118, 119, 132, 133, 137, 138, 195, 196, 197, 198, 248, 250, 253, 255],
    profiles=[4,15,16],
    Staad_objects=STAAD_objects
)

vertical_bracing = member_group(
    id='vertical_bracing',
    members=[29, 30, 31, 32, 33, 34, 35, 36, 141, 142, 143, 144, 145, 146, 147, 148],
    profiles=[4,15,16],
    Staad_objects=STAAD_objects
)

'''Bracket'''
bracket_beams = member_group(
    id='bracket_beams',
    members=[335, 336, 337, 338, 340, 341],
    profiles=[10,5,8],
    Staad_objects=STAAD_objects
)

bracket_bracing = member_group(
    id='bracket_bracing',
    members=[339, 342, 343, 344, 345, 346, 347],
    profiles=[3,9],
    Staad_objects=STAAD_objects,
)

optimise_group_collection = optimise_groups().add(tie_beams_group_1)

In [ ]:
'''Optimiser function'''
group:member_group
for group in optimise_group_collection.member_groups:

    print(f'\n\nOptimising member group {group.id} for {len(group.profiles)} profiles : {group.profiles}')

    # set all members to their initial profile
    print('Setting default profile for all groups')
    for group_i in optimise_group_collection.member_groups:
        group_i.set_members_property_initial()

    # loop through all profiles in a group : assign , analyse and choose the best profile
    for index in range(len(group.profiles)):
        print(f'\nProfile id : {group.profiles[index]}')

        group.set_members_property(index)
        run_analysis(openSTAAD)

        group.get_utilization_ratios_for_profile(index)
        
        print('Group Result')
        print(group.results[index])
        print(group.get_failed_members_for_profile(index))

        all_members = member_group(
            id='whole_structure',
            members=beam_nos,
            exclude_members=group.members,
            Staad_objects=STAAD_objects,
        )
        all_members.get_utilization_ratios()

        print('Model Result')
        print(all_members.results)
        print(all_members.get_failed_members())

'''Reset profiles'''
for group_i in optimise_group_collection.member_groups:
    group_i.set_members_property_initial()

In [ ]:
'''Reset profiles'''
for group_i in optimise_group_collection.member_groups:
    group_i.set_members_property_initial()

In [ ]:
'''Group selector 1 in STAAD UI'''
# columns_group_1.select_members()

# beams_tier_1.select_members()
# beams_tier_2.select_members()
# beams_tier_3.select_members()

# bracket_beams.select_members()

In [ ]:
'''Group selector 2 in STAAD UI'''
# int_beams_tier_2.select_members()
tie_beams_group_1.select_members()
# stub_columns_group_1.select_members()

In [ ]:
'''Group selector 3 in STAAD UI'''
# plan_bracing.select_members()
# vertical_bracing.select_members()
# bracket_bracing.select_members()

In [ ]:
run_analysis(openSTAAD)

all_members = member_group(
            id='whole_structure',
            members=beam_nos,
            Staad_objects=STAAD_objects,
            allowable_ratio=0.85
        )
all_members.get_utilization_ratios()

print('Model Result')
print(all_members.results)
print(all_members.get_failed_members())